In [5]:
import pygame
import random
import math
import pickle


# RGB colour definitions for referring to later
black = (0, 0, 0)
white = (255, 255, 255)
grey = (100, 100, 100)
darkGrey = (50, 50, 50)
light_grey = (130, 130, 130)


# Base/parent class used for all other classes
# Should be treated as abstract - there should never be an Element object, only objects that are children of Element
class Element:

    # x, y = the x and y position of the top left of the element in pixels
    # width, height = width + height of the element in pixels
    # font = The Pygame Font object used for rendering text
    # bg_colour = The colour of background parts of the element as an RGB tuple
    # text_colour = The colour of text of the element as an RGB tuple
    def __init__(self, x, y, width, height, font, back_colour=grey, text_colour=black):
        # x and y can be a decimal value as these are not the values used in drawing
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        # Pygame Rect object that covers the entire object, used for collision detection with mouse
        self.rect = pygame.Rect(self.x, self.y, self.width, self.height)
        # x2 and y2 are the co-ords for the bottom right of the element
        self.x2 = self.x + self.width
        self.y2 = self.y + self.height
        self.font = font
        self.bg_colour = back_colour
        self.text_colour = text_colour

    @property
    def bg_colour(self):
        return self._bg_colour

    # Validation check before setting background colour to new value
    # Prevents crash due to invalid colour where one element is greater than 255 or less than 0
    @bg_colour.setter
    def bg_colour(self, new_colour):
        valid = True
        for n in new_colour:
            if n > 255 or n < 0:
                valid = False
        if valid:
            self._bg_colour = new_colour

    # Default methods, child classes override the ones they need
    # Uses 'pass' keyword: method does nothing

    # Default draw method
    # Parameter screen is a Pygame surface object that will be drawn to
    def draw(self, screen):
        pass

    # Method that deals with clicking input, takes in the mouse position as 2 co-ords
    def on_click(self, mouse_x, mouse_y):
        pass

    # Method that deals with mouse button being released
    def on_unclick(self):
        pass

    # Method that deals with a keyboard key being pressed
    # Takes in the pygame key code as a parameter
    def on_char_typed(self, key_pressed):
        pass

    # Method that deals with a keyboard key being released
    # Takes in the pygame key code as a parameter
    def on_key_up(self, key_up):
        pass

    # Method for things that should be run once a frame
    # Takes in the mouse pos as 2 co-ords as parameters
    def update(self, mouse_x, mouse_y):
        pass

    # Method that is called when an Element object is added to a Menu or Group object
    # For explanation, see methods where overriden
    def on_menu_add(self):
        pass


# Class for a drop-down list that displays a list of pre-defined options
# Inherits all methods and attributes from Element
class DropDown(Element):

    # Static constant for how wide the button at the side of the list should be
    buttonWidth = 30

    # data = A list of possible options - strings
    # font = The pygame Font object used to render text
    def __init__(self, x, y, width, height, data, font):
        # Calls its parent's init method to get all parent attributes
        Element.__init__(self, x, y, width, height, font)
        self.bg_colour = light_grey
        self.data = data
        self.current_opt = 0
        self.button_text = self.font.render(self.data[self.current_opt], 1, black)
        # Make text objects for all data objects
        self.options = data
        # Open is a boolean that tracks whether the list should be drawn
        self.open = False
        # Pygame Rect object that covers the button
        self.button_rect = pygame.Rect(self.x2, self.y, DropDown.buttonWidth, self.height)
        # Pygame Rect object that covers the menu
        self.menu_rect = pygame.Rect(self.x, self.y2, self.width, self.height*len(self.data))

    def on_menu_add(self):
        self.button_rect = pygame.Rect(self.x2, self.y, DropDown.buttonWidth, self.height)
        self.menu_rect = pygame.Rect(self.x, self.y2, self.width, self.height*len(self.data))

    def on_click(self, mouse_x, mouse_y):
        # Returns true if an option changed
        changed = False
        # Checks if the menu is open
        if self.open:
            # Checks if clicking button
            if self.button_rect.collidepoint(mouse_x, mouse_y):
                # Closes the drop down menu
                self.open = False
            # If clicking the menu, select the option they clicked on, then close the menu
            if self.menu_rect.collidepoint(mouse_x, mouse_y):
                self.select_option(mouse_y)
                self.open = False
                # Option has been changed
                changed = True
        else:
            # Checks if clicking button
            if self.button_rect.collidepoint(mouse_x, mouse_y):
                # Open the drop down menu
                self.open = True
        return changed

    # Using property modifier for getter and setter
    @property
    def options(self):
        return self.__options

    # Uses setter to make sure when options change, text objects are automatically created
    # Takes in a list of strings as a parameter
    @options.setter
    def options(self, data):
        options = []
        # For each string in data, make a text object from it
        for i in range(len(data)):
            text = self.font.render(data[i], 1, black)
            options.append(text)
        self.__options = options
        # Recreates the collision Rect object to account for longer menu box
        self.menu_rect = pygame.Rect(self.x, self.y2, self.width, self.height * (len(self.data)))

    # Takes in the y co-ord of the mouse
    # Subtracts from the y co-ord so the top of the first option box is at 0
    # Divides by the height of each option box then rounds it down
    def select_option(self, mouse_y):
        self.current_opt = math.floor((mouse_y - self.y - self.height) / self.height)
        # Changes the button text to the currently selected option
        self.change_text(self.data[self.current_opt])

    # Changes the text in the button to string new_text
    def change_text(self, new_text):
        self.button_text = self.font.render(new_text, 1, black)

    # Draws the drop-down box
    def draw(self, screen):
        # Draws the background of the box
        pygame.draw.rect(screen, self.bg_colour, (self.x, self.y, self.width, self.height))
        # Draws the background for the button next to the box
        pygame.draw.rect(screen, darkGrey, ((self.x + self.width), self.y, DropDown.buttonWidth, self.height))
        # Draws the triangle inside the button
        pygame.draw.polygon(screen, black, (((self.x + self.width + (DropDown.buttonWidth / 2)),
                                             (self.y + self.height - 3)), ((self.x + self.width + 3), (self.y + 3)),
                                            ((self.x2 + DropDown.buttonWidth - 3), (self.y + 3))))
        # Draw text in box
        screen.blit(self.button_text, (self.x + 2, self.y + 2))
        # Draw border around box
        pygame.draw.lines(screen, black, True, ((self.x, self.y), (self.x2, self.y), (self.x2, self.y2), (self.x, self.y2)))
        # Displays whole list if open
        if self.open:
            # For each option available, draw a box with text in
            for i in range(len(self.data)):
                current_y = self.y + ((i+1)*self.height)
                # Render a box
                pygame.draw.rect(screen, self.bg_colour, (self.x, current_y, self.width, self.height))
                # Render the text
                screen.blit(self.options[i], (self.x + 2, current_y + 2))


# Class for a button with a text label
# Inherits all methods and attributes from Element
class Button(Element):

    # text = The text rendered as the button's label
    def __init__(self, x, y, font, text):
        self.text = text
        # Width and Height are generated based on the width and height of the text
        self.width = font.size(text)[0] + 5
        self.height = font.size(text)[1] + 5
        Element.__init__(self, x, y, self.width, self.height, font)
        self.bg_colour = light_grey
        # Makes a text object of the label text
        self.txt_obj = self.font.render(self.text, 1, self.text_colour)
        # Clicked is a boolean value which is true when the user has clicked on the button
        self.clicked = False
        # The number of frames since the button was last clicked
        self.last_click = 0
        # The width of the black border around the button in pixels
        self.border = 1
        # When this is true, the button appears greyed out and cannot be clicked
        self.grey = False

    # Using getters and setters for attribute 'grey'
    @property
    def grey(self):
        return self._grey

    # Sets grey_change to true when grey has been changed
    # The part of update that deals with colour should only be run once, not on every update
    @grey.setter
    def grey(self, new_grey):
        self._grey = new_grey
        self.update_grey()

    # When mouse button released, clicked = false
    def on_unclick(self):
        self.clicked = False

    # When mouse clicked, checks if mouse is inside button
    # Checks if button has not been pressed in last 20 frames
    # Checks if button is not greyed out
    # If all True, button is clicked and last_click set to 20
    def on_click(self, mouse_x, mouse_y):
        if self.rect.collidepoint(mouse_x, mouse_y)and self.last_click == 0 and not self.grey:
            self.clicked = True
            self.last_click = 20

    # Called every frame, checks if mouse is inside button but doesn't need to be clicked
    def on_hover(self, mouse_x, mouse_y):
        # If in button, make border thicker and make background slightly lighter
        if self.rect.collidepoint(mouse_x, mouse_y) and not self.grey:
            self.border = 2
            self.bg_colour = (100, 100, 100)
        # If not in button, set border and colour back to normal
        else:
            self.border = 1
            self.bg_colour = light_grey

    # Called every second
    def update(self, mouse_x, mouse_y):
        # Runs method to check if mouse is inside button
        self.on_hover(mouse_x, mouse_y)
        # If button has not been clicked in that frame, decrement button counter and set clicked to false
        if self.last_click != 0:
            self.last_click -= 1
            self.clicked = False

    def update_grey(self):
        # If the button is greyed out, make background colour darker
        if self.grey:
            self.bg_colour = (150, 150, 150)
            # Try to make text colour darker
            # Uses try statement because Button is parent class of ImageButton
            # ImageButton has no text attribute
            try:
                self.text_colour = darkGrey
                self.txt_obj = self.font.render(self.text, 1, self.text_colour)
            except AttributeError:
                pass
        # If not grey, set background colour and text colour to normal
        else:
            self.bg_colour = light_grey
            try:
                self.text_colour = black
                self.txt_obj = self.font.render(self.text, 1, self.text_colour)
            except AttributeError:
                pass
        
    def draw(self, screen):
        # Draws the background rectangle of the button
        pygame.draw.rect(screen, self.bg_colour, (self.x, self.y, self.width, self.height))
        # Draws the button text
        screen.blit(self.txt_obj, (self.x + 3, self.y + 3))
        # Draws the border
        pygame.draw.lines(screen, black, True, ((self.x, self.y), (self.x, self.y2), (self.x2, self.y2),
                                                    (self.x2, self.y)), self.border)


# Child of the button class but displays an image instead of a text label
# Inherits all methods and attributes from Button which also inherits from Element
class ImageButton(Button):

    # Takes in a filepath to an image instead of text label
    def __init__(self, x, y, font, filepath):
        # Tries to open the image specified by 'filepath' in the /img folder
        # The root of the /img folder is the folder where this .py file is
        try:
            self.image = pygame.image.load("img/" + filepath + ".png")
        # Validation: Tell user if image cannot be found
        except FileNotFoundError:
            print("Could not find file at img/" + filepath + ".png")
        # Get width and height of image
        size = self.image.get_rect().size
        # Dimensions of button is dimensions of image with 10 pixels of padding in each direction
        self.width = size[0] + 10
        self.height = size[1] + 10
        Element.__init__(self, x, y, self.width, self.height, font)
        self.border = 1
        self.clicked = False
        self.last_click = 0
        self.grey = False

    def draw(self, screen):
        # Draw the background
        pygame.draw.rect(screen, self.bg_colour, (self.x, self.y, self.width, self.height))
        # Draw the image
        screen.blit(self.image, (self.x + 5, self.y + 5))
        # Draw the borders
        pygame.draw.lines(screen, black, True, ((self.x, self.y), (self.x, self.y2), (self.x2, self.y2),
                                                    (self.x2, self.y)), self.border)


# Class for a slider that has a small triangle that moves along a bar when clicked and dragged
# The output is always between 2 limits, given by parameter limits, a tuple of length 2
# Lower limit is limit[0], upper limit is limit[1]
# starting_pos determines how far along the line the pointer starts where 0 is fully left
# 1 is fully right and 0.5 is halfway. Defaults to 0.5
# dec_points - how many decimal points the text should render, defaults to 0 (integers only)
# Inherits all methods and attributes from Element
class Slider(Element):

    # Limits = the lowest and highest points given as a tuple
    # StartingPos = how far along the bar the pointer is at when program starts
    def __init__(self, x, y, width, height, font, limits, starting_pos=.5, dec_points = 0):
        Element.__init__(self, x, y, width, height, font)
        self.limits = limits
        # line_y = The y value at which the line starts
        self.line_y = self.y + (self.height * 0.8)
        self.starting_pos = starting_pos
        self.dec_points = dec_points
        # 'pointer' is the raw pixel position of the x co-ord of the middle of the triangular pointer
        self.pointer = self.x + (self.width * self.starting_pos)
        # Value is the output of the slider
        self.value = self.get_pos()
        # txt is the text object that renders the value of the slider
        self.txt = self.update_txt()
        # true when the slider itself is clicked
        self.clicked = False
        # true when the pointer is clicked
        self.tri_clicked = False
        # Pygame Rect object for the triangle pointer
        self.tri_rect = pygame.Rect(self.pointer - 10, self.y + 2, 20, (self.line_y - 2) - (self.y + 2))

    # Called when a slider object is added to a Menu object
    # Updates all x and y positions of the triangle and text
    def on_menu_add(self):
        self.line_y = self.y + (self.height * 0.8)
        self.pointer = self.x + (self.width * self.starting_pos)
        self.value = self.get_pos()
        self.rect = pygame.Rect(self.x, self.y, self.width, self.height)
        self.tri_rect = pygame.Rect(self.pointer - 10, self.y + 2, 20, (self.line_y - 2) - (self.y + 2))
        self.update_txt()

    # Given the raw pointer position relative to the top left corner of the screen
    # Gets the value from the slider and returns it
    def get_pos(self):
        # Gets the proportion of slider to left of pointer
        # Eg. if 10% of slider to left of pointer, result is 0.1
        pos = (self.pointer - self.x) / self.width
        # Multiplies proportion by the difference between the limits
        # This gives a proportional value of how far the pointer is from the left limit
        pos = pos * (self.limits[1] - self.limits[0])
        # Adds the lower limit
        pos += self.limits[0]
        return pos

    # Updates the text object of the value above the pointer
    def update_txt(self):
        if self.dec_points == 0:
            txt = self.font.render(str(round(self.value)), 1, black)
        else:
            txt = self.font.render(str(round(self.value, self.dec_points)), 1, black)
        return txt

    def draw(self, screen):
        # Draws bottom line
        pygame.draw.rect(screen, black, (self.x, self.line_y, self.width, self.y2 - self.line_y))
        # Draws triangular pointer 2 pixels above the line
        pygame.draw.polygon(screen, black, ((self.pointer, self.line_y - 2), (self.pointer - 10, self.y + 2),
                                                (self.pointer + 10, self.y + 2)))
        # Draws value above pointer
        self.value = self.get_pos()
        self.txt = self.update_txt()
        screen.blit(self.txt, (self.pointer + 12, self.y))

    # If clicked and is in bounds of the triangle, clicked = True
    def on_click(self, mouse_x, mouse_y):
        if self.tri_rect.collidepoint(mouse_x, mouse_y):
            self.tri_clicked = True
        elif self.rect.collidepoint(mouse_x, mouse_y):
            self.clicked = True

    # sets clicked booleans to false when mouse button released
    def on_unclick(self):
        self.tri_clicked = False
        self.clicked = False

    # Run every frame. Only needs x co-ord of mouse
    # Requires both co-ords but sets y to None as default to allow overriding of method of same name in Element
    def update(self, mouse_x, mouse_y=None):
        if self.tri_clicked or self.clicked:
            # If mouse x co-ord further than upper boundary
            if mouse_x > self.x2:
                # Pointer = upper boundary
                self.pointer = self.x2
            # If mouse x co-ord further than lower boundary
            elif mouse_x < self.x:
                # Pointer = lower boundary
                self.pointer = self.x
            # Otherwise, mouse x co-ord is between the 2 boundaries
            # pointer = mouse x co-ord
            else:
                self.pointer = mouse_x
            self.tri_rect = pygame.Rect(self.pointer - 10, self.y + 2, 20, (self.line_y - 2) - (self.y + 2))


# Class for a text entry box
# Inherits all methods and attributes from Element
class Textbox(Element):

    # Static tuple of pygame character codes that have a different key_name than a letter
    # Eg. key_name of the g key is g, therefore not included in tuple
    special_chars = ("space", "escape", "left ctrl", "right ctrl", "return", "left alt", "right alt", "caps lock",
                     "numlock", "scroll lock", "tab", "left super", "right super", "menu", "f1", "f2", "f3", "f4", "f5",
                     "f6", "f7", "f8", "f9", "f10", "f11", "f12", "insert", "home", "delete", "end", "page up",
                     "page down", "pause")
    # Dictionary of the keys that have different characters when shift is pressed with them
    # and the corresponding characters
    shifts = {"1": "!", "2": "\"", "3": "£", "4": "$", "5": "%", "6": "^", "7": "&", "8": "*", "9": "(", "0": ")",
              "-": "_", "=": "+", "#": "~", "[": "{", "]": "}", ";": ":", "'": "@", ",": "<", ".": ">", "/": "?",
              "\\": "|"}
    # Just gets the keys from the shifts dictionary
    # tuple of numbers 0 - 9
    shift_keys = shifts.keys()
    # Static variable that holds the total number of text boxes in the program
    # Used in checking which text box is in focus
    # A text box must be in focus in order to register keyboard input
    TextBoxes = 0
    
    # blocked_chars = the characters the box will not accept
    # char_limit = the maximum number of characters allowed in the textbox
    def __init__(self, x, y, width, height, font, blocked_chars, char_limit = None):
        Element.__init__(self, x, y, width, height, font)
        self.blocked_chars = blocked_chars
        self.charLimit = char_limit
        # text is a string that holds the characters input to the text box
        self.text = ""
        # txt_obj is a text object used for rendering the input
        self.txt_obj = font.render(self.text, 1, black)
        self.update_text()
        # Adds one to the count of text boxes
        Textbox.TextBoxes += 1
        # By default is not in focus
        self.is_focused = False
        # If it is the only text box, automatically in focus
        if Textbox.TextBoxes < 2:
            self.is_focused = True
        # Boolean value of whether the shift key is held down
        self.shift_pressed = False

    # If the user clicks on the text box, it is in focus
    # If the user hasn't clicked on the text box, defocuses it to prevent multiple text boxes in focus at one time
    def on_click(self, mouse_x, mouse_y):
        if self.rect.collidepoint(mouse_x, mouse_y):
            self.is_focused = True
        else:
            self.is_focused = False

    # Called every time a key is pressed
    def on_char_typed(self, key_pressed):
        # Only runs the code if the textbox is in focus
        if self.is_focused:
            # Special case for backspace, removes the last letter of the string
            if key_pressed == pygame.K_BACKSPACE:
                self.text = self.text[:-1]
            # Checks if shift key is pressed
            elif key_pressed == pygame.K_LSHIFT or key_pressed == pygame.K_RSHIFT:
                self.shift_pressed = True
            # Checks the character limit has not been reached
            elif len(self.text) < self.charLimit:
                # is_allowed is True by default, if the key input is a blocked char, it become false
                is_allowed = True
                # is_special is False by default
                # If key input is a special character (from special_chars list) then it is True
                is_special = False
                # key_name gets the name of the corresponding pygame key code
                # for alphanumeric characters, is the same as the character itself
                key_name = pygame.key.name(key_pressed)
                # Checking if character is blocked
                for c in self.blocked_chars:
                    if key_name == c:
                        is_allowed = False
                        break
                # Checking if character is special
                for s in Textbox.special_chars:
                    if key_name == s:
                        is_special = True
                        break
                # If the character is allowed and isn't a special character, it can be added normally
                if is_allowed and not is_special:
                    # If the shift key is being held down
                    if self.shift_pressed:
                        # is_shift_key is False by default but becomes true if
                        # the key being pressed is the shift_keys list
                        is_shift_key = False
                        # Checks through the shift_keys list for the key being pressed
                        for k in Textbox.shift_keys:
                            if key_name == k:
                                is_shift_key = True
                        # If the key is in shift_key, use that key's shift equivalent
                        # Add it to the text string
                        if is_shift_key:
                            self.text = self.text + Textbox.shifts[key_name]
                        # If not in shift_keys, just add the uppercase equivalent of the letter typed
                        else:
                            self.text = self.text + key_name.upper()
                    # If shift key not pressed, just add the character typed to text
                    else:
                        self.text = self.text + key_name
                # If it's an allowed character and is a special character
                # Special cases are dealt with here
                elif is_allowed and is_special:
                    # Key name of the space bar is not ' ', must be manually checked for
                    if key_pressed == pygame.K_SPACE:
                        self.text = self.text + " "
            # Updates the text object being drawn to the screen
            self.update_text()

    # Called every time a key is released
    def on_key_up(self, key_up):
        # Checks if shift is unpressed
        if key_up == pygame.K_RSHIFT or key_up == pygame.K_LSHIFT:
            self.shift_pressed = False

    def draw(self, screen):
        # Draws white background box
        pygame.draw.rect(screen, white, (self.x, self.y, self.width, self.height))
        # Draws outline if focused
        if self.is_focused:
            pygame.draw.lines(screen, black, True, ((self.x, self.y), (self.x + self.width, self.y),
                                                        (self.x + self.width, self.y + self.height),
                                                        (self.x, self.y + self.height)), 2)
        # Draws text if not empty
        if self.text != "":
            screen.blit(self.txt_obj, (self.x + 2, self.y + 2))

    # Called when the text in the text box is updated, recreates the text object
    def update_text(self):
            self.txt_obj = self.font.render(self.text, 1, black)

    @property
    def text(self):
        return self._text

    # Method for externally setting the text in the text box
    # changes text to the parameter new_text then updates the text object
    @text.setter
    def text(self, new_text):
        self._text = new_text
        self.update_text()


# Colour patch draws a rectangle of a specific colour on the screen
# Useful for showing output of RGB selectors
# Inherits all methods and attributes from Element
class ColourPatch(Element):

    # rgb - A tuple of 3 integers between 0 and 255 to represent a 24 bit colour
    def __init__(self, x, y, width, height, font, rgb):
        Element.__init__(self, x, y, width, height, font)
        self.rgb = rgb

    @property
    def rgb(self):
        return self._rgb

    # Setter method acts as validation to make sure an RGB colour does not contain numbers outside of 0-255
    @rgb.setter
    def rgb(self, new_rgb):
        valid = True
        for num in new_rgb:
            if num > 255 or num < 0:
                valid = False
        if valid:
            self._rgb = new_rgb
        else:
            print("RGB colour must be between 0 and 255")
    
    def draw(self, screen):
        # Draws the colour
        pygame.draw.rect(screen, self.rgb, (self.x, self.y, self.width, self.height))
        # Draws a border
        pygame.draw.lines(screen, black, True, ((self.x, self.y), (self.x2, self.y), (self.x2, self.y2),
                                                    (self.x, self.y2)))


# A Group is a list of Elements that can be addressed all at once
# Inherits all methods and attributes from Element
# Uses a list to contain all elements contained within it
# Rather than calling the update method of every element in the list,
# you can just call the update method of the group, which calls them all
# as all Element objects have an update method
class Group(Element):

    def __init__(self, x, y, width, height, font):
        Element.__init__(self, x, y, width, height, font)
        # visible - whether to draw the elements or not
        self.visible = False
        # The list of elements
        # Any object that inherits from the Element class can be added
        # The elements in the list are associated with the Menu object but are not deleted if the menu is deleted
        self.elements = []
        # texts is a two-dimensional list that stores pygame text objects
        # and the co-ords where each object should be drawn
        self.texts = [[], []]

    # Adds an element (any object that inherits the Element class) to the group
    def add(self, element):
        try:
            # Adds the x and y co-ords of the group to the element's co-ord
            element.x += self.x
            element.y += self.y
            element.x2 += self.x
            element.y2 += self.y
            element.rect.move_ip(self.x, self.y)
            # Run method that allows objects to do things specific to them when added
            element.on_menu_add()
            # Add object to elements list
            self.elements.append(element)
        except AttributeError:
            print("Error: Tried adding a non-element object to a group")

    # Adds text to render in the group, takes 2 parameters
    # text - the text to be added, in string form
    def add_text(self, text, coords):
        self.texts[0].append(self.font.render(text, 1, (255, 0, 0)))
        self.texts[1].append((coords[0] + self.x, coords[1] + self.y))

    def draw(self, screen):
        if self.visible:
            # Draws each element in the group by calling its draw method
            for element in self.elements:
                element.draw(screen)
            # Draws each text object in the group to the screen
            for i in range(len(self.texts[0])):
                screen.blit(self.texts[0][i], self.texts[1][i])

    def on_click(self, mouse_x, mouse_y):
        # Runs each element's on_click method
        for element in self.elements:
            element.on_click(mouse_x, mouse_y)

    def on_unclick(self):
        # Runs each element's on_unclick method
        for element in self.elements:
            element.on_unclick()

    def on_char_typed(self, key_pressed):
        # Runs each element's on_char_typed method
        for element in self.elements:
            element.on_char_typed(key_pressed)

    def on_key_up(self, key_up):
        for element in self.elements:
            element.on_key_up(key_up)

    def update(self, mouse_x, mouse_y):
        for element in self.elements:
            element.update(mouse_x, mouse_y)


# Child of the Group class
# Inherits all methods and attributes from Group and therefore from Element
# Use of polymorphism: uses same methods as parent Group
# Functions exactly like a group but looks like a menu
# Has background and a top bar similar to a normal window
class Menu(Group):

    # title is a string that is drawn on the top bar
    # optional parameter bar_height sets the height of the top bar
    def __init__(self, x, y, width, height, font, title, bar_height=25):
        Group.__init__(self, x, y, width, height, font)
        self.title = title
        self.bar_height = bar_height
        self.total_height = self.height + self.bar_height
        # y3 is the y co-ord where the main part of the menu starts and the top bar ends
        self.y3 = self.y + self.bar_height
        # Creates a pygame text object for the title
        self.txt = self.font.render(title, 1, black)

    # Adds an element (any object that inherits the Element class) to the menu
    # Overrides Group add method to factor in height of menu bar
    def add(self, element):
        try:
            # Adds the x and y co-ords to the element's co-ord so it appears on the menu
            element.x += self.x
            element.y += (self.y + self.bar_height)
            element.x2 += self.x
            element.y2 += (self.y + self.bar_height)
            element.rect.move_ip(self.x, self.y + self.bar_height)
            # Run method that allows objects to do things when added to a Group or menu
            element.on_menu_add()
            # Add to the elements list
            self.elements.append(element)
        except AttributeError:
            print("Error: Adding a non-element object")

    # Adds text to render in the menu
    # Overrides Group addText method to factor in height of menu bar
    def add_text(self, text, coords):
        self.texts[0].append(self.font.render(text, 1, black))
        self.texts[1].append((coords[0] + self.x, coords[1] + self.y + self.bar_height))

    def draw(self, screen):
        if self.visible:
            # Draw top bar of menu
            pygame.draw.rect(screen, (80, 80, 80), (self.x, self.y, self.width, self.bar_height))
            # Draw title on top bar
            screen.blit(self.txt, (self.x+2, self.y+2))
            # Draw bg of menu
            pygame.draw.rect(screen, (120, 120, 120), (self.x, self.y3, self.width, self.height))
            # Draw border of menu
            pygame.draw.lines(screen, black, True, ((self.x, self.y), (self.x, self.y2 + self.bar_height),
                                                        (self.x2, self.y2 + self.bar_height), (self.x2, self.y)))
            pygame.draw.line(screen, black, (self.x, self.y3), (self.x2, self.y3))
            # Draws each element in the group by calling its draw method
            for element in self.elements:
                element.draw(screen)
            # Draws each text object in the group to the screen
            for i in range(len(self.texts[0])):
                screen.blit(self.texts[0][i], self.texts[1][i])


# Checkbox is a small box which when clicked will toggle between an 'on' and 'off' state
# Inherits all methods and attributes from Element
class Checkbox(Element):

    # Width and height are both set to 22 by default though can be changed
    # off_img and on_img are the images used for the off state and on state respectively
    def __init__(self, x, y, font, width=22, height=22, off_img="dan_gui/checkboxOff", on_img="dan_gui/checkboxOn"):
        Element.__init__(self, x, y, width, height, font)
        self.offImg = pygame.image.load("img/" + off_img + ".png")
        self.onImg = pygame.image.load("img/" + on_img + ".png")
        # Checkbox is off by default
        self.on = False

    def draw(self, screen):
        # Draw background rectangle
        pygame.draw.rect(screen, self.bg_colour, (self.x, self.y, self.width, self.height))
        # Draw border
        pygame.draw.lines(screen, black, True, ((self.x, self.y), (self.x, self.y2), (self.x2, self.y2),
                                                    (self.x2, self.y)), 2)
        # If on, draw the 'on' image
        if self.on:
            screen.blit(self.onImg, (self.x + 2, self.y + 2))
        # If off, draw the 'off' image
        else:
            screen.blit(self.offImg, (self.x + 2, self.y + 2))

    # If clicked on, toggle between on and off state
    def on_click(self, mouse_x, mouse_y):
        if self.rect.collidepoint(mouse_x, mouse_y):
            if self.on:
                self.on = False
            else:
                self.on = True


# Class for a rectangle that is drawn to the screen and has a collision hit box
# Represents a metal terminal
class MetalRect:

    # Takes x, y, width and height as parameters
    # Used in drawing the rectangle
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        # Creates a pygame Rect object to manage collisions
        self.rect = pygame.Rect(self.x, self.y, self.width, self.height)

    # Draws the rectangle to the screen
    def draw(self, screen, colour):
        pygame.draw.rect(screen, colour, (self.x, self.y, self.width, self.height))


# Class that models the behaviour of a photon
class Photon:

    # All photon objects are held in this static one-dimensional list
    PhotonList = []
    # Constant value for radius of each photon in pixels
    Radius = 4
    # Static value that keeps track of how many frames it has been since the last photon was emitted
    LastEmitted = 0

    # Photon object takes a tuple of 3 integers between 0 and 255 as the colour
    # Also takes a real number as the kin_energy to represent the kinetic energy of the photon
    def __init__(self, colour, kin_energy):
        self.colour = colour
        self.kinEnergy = kin_energy
        # Randomises x and y co-ords along the bottom of the lamp image
        self.x = 500 + 16 + random.randint(0, 180)
        self.y = 150 + 54 + random.randint(0, 100)
        # the speed variables represent how many pixels the photon moves in each axis per frame
        self.h_speed = -10
        self.v_speed = 4
        # Creates a pygame Rect object to handle collisions with metal terminal
        self.rect = pygame.Rect(self.x, self.y, 2*Photon.Radius, 2*Photon.Radius)

    # Destroys the object by removing itself by the list then deleting itself
    def destroy(self):
        index = self.find_self()
        Photon.PhotonList.pop(index)
        del self

    # Allows a photon to find itself in the PhotonList by comparing itself to each item in the list
    # Returns the index of that photon in the PhotonList
    def find_self(self):
        index = -1
        for i in range(len(Photon.PhotonList)):
            if Photon.PhotonList[i] == self:
                index = i
                break
        return index

    # Alters the x and y co-ords of the photon by the respective speed variable
    def move(self):
        self.x += self.h_speed
        self.y += self.v_speed
        # Moves the pygame Rect object for collisions
        self.rect.move_ip(self.h_speed, self.v_speed)

    # Checks if the photon object's collision Rect collides with the parameter rect
    # Takes stop_voltage for should_create_electron
    # If collision detected, checks if electron should be made
    # Electron object made if necessary, then photon is deleted
    def check_collision(self, rect, stop_voltage):
        if self.rect.colliderect(rect):
            if self.should_create_electron(stop_voltage):
                self.create_electron()
            self.destroy()
        # If photon goes off screen (either too far left or too far right) then it is deleted
        elif self.x < -2*Photon.Radius or self.y > 800 + 2*Photon.Radius:
            self.destroy()

    # Creates an electron object with the same y co-ord and kinetic energy
    def create_electron(self):
        Electron.ElectronList.append(Electron(self.y, self.kinEnergy))

    # If the kinetic energy of the photon (minus stopping voltage) is greater than 0, returns true
    def should_create_electron(self, stop_voltage):
        stop_voltage = stop_voltage * 1.6 * math.pow(10, -19)
        if (self.kinEnergy - stop_voltage) > 0 * math.pow(10, -19):
            # Only affects actual variable once electron is about to be made
            # Prevents stopping voltage being taken away multiple times
            self.kinEnergy = self.kinEnergy - stop_voltage
            return True
        else:
            return False

    # Draws a circle to the screen to represent the photon
    def draw(self, screen):
        pygame.draw.circle(screen, self.colour, (self.x, self.y), Photon.Radius)


# Class to model an electron particle
class Electron:

    # The one-dimensional list of all electrons between the 2 metal plates
    ElectronList = []
    # The one-dimensional list of all electrons that have hit the right metal plate in the last second
    # Constant value used in drawing the circle that represents the electron
    Radius = 5
    # Constant value for the mass of an electron
    Mass = 9.11 * math.pow(10, -31)

    # Takes in the y co-ord and kinetic energy as parameters
    def __init__(self, y, kin_energy):
        self.kinEnergy = kin_energy
        self.x = 60
        self.y = y
        self.draw_x = round(self.x)
        self.draw_y = round(self.y)
        # Creates a pygame Rect object to handle collisions
        self.rect = pygame.Rect(self.draw_x, self.draw_y, 2 * Electron.Radius, 2 * Electron.Radius)
        # Gets the speed of the electron in pixels per frame by multiplying it by 10^19
        self.speed = kin_energy * math.pow(10, 19)

    # Destroys electron by removing from ElectronList then deleting it
    def destroy(self):
        index = self.find_self()
        Electron.ElectronList.pop(index)
        del self

    # Finds self in ElectronList by comparing each object to itself then returns the index
    def find_self(self):
        index = -1
        for i in range(len(Electron.ElectronList)):
            if Electron.ElectronList[i] == self:
                index = i
                break
        return index

    # Changes the x co-ord and the top-left co-ord of the Rect of the electron by its speed
    # Only moves in x axis, electrons moving horizontally only.
    def move(self):
        self.x += self.speed
        self.draw_x = round(self.x)
        self.rect = pygame.Rect(self.draw_x, self.draw_y, 2 * Electron.Radius, 2 * Electron.Radius)

    # If the electron is colliding with the Rect parameter rect
    # Deletes electron object
    def check_pos(self, rect):
        if self.rect.colliderect(rect):
            self.destroy()

    # Draws a circle to represent the electron
    def draw(self, screen):
        # Draw inner part
        pygame.draw.circle(screen, (60, 230, 255), (self.draw_x, self.draw_y), Electron.Radius - 1)
        # Draw border
        pygame.draw.circle(screen, (0, 0, 0), (self.draw_x, self.draw_y), Electron.Radius, 2)


# Class to represent a metal
class Metal:

    # Static list of metal objects
    MetalList = []
    # Static list of the names of all metal objects
    MetalNames = []

    # Parameters:
    # name - The Metal's name
    # work_func - The work function of the metal
    # colour - an tuple of 3 ints from 0-255 to represent an RGB colour
    def __init__(self, name, work_func, colour):
        self.name = name
        self.work_func = work_func
        self.colour = colour
        # On Initialisation adds the metal's name to a list of metal names
        Metal.MetalNames.append(name)


# Beginning of actual code
# Initialise all pygame modules before they can be used
pygame.init()

# These variables hold the dimensions of the screen, should be kept constant
display_width = 800
display_height = 600

# Colour definitions for referring to later
black = (0, 0, 0)
white = (255, 255, 255)
grey = (100, 100, 100)
lightGrey = (180, 180, 180)

# Initialise main drawing surface
screen = pygame.display.set_mode((display_width, display_height))
# Set title of window
pygame.display.set_caption("Photoelectric Effect Simulator")
# Create clock object for timing
clock = pygame.time.Clock()

# Tuple of min wavelengths for UV, violet, blue, cyan, yellow and red
wlValues = (850, 750, 620, 570, 495, 450, 380, 0)
wlValues2 = (0, 380, 450, 495, 570, 620, 750, 850)


# Basic method to convert a string to an integer
def get_int_from_str(text):
    # Try statement catches errors in case of invalid input
    try:
        i = int(text)
    except ValueError:
        i = 0
    return i


# Basic method to convert a string to a float
def get_float_from_string(text):
    # Try statement catches error in case of invalid input
    try:
        i = float(text)
    except ValueError:
        i = 0
    return i


# Called 30 times a second to check if an photon should be emitted
def emit_photon(current_metal, intensity, wavelength):
    # firstly checks if intensity is above 0, if not, no photons are being released
    if intensity > 0:
        # Photon.LastEmitted is a timer, whenever it reaches 0, a photon should be emitted
        if Photon.LastEmitted == 0:
            # Creates frequency, needed for calculations
            frequency = (3 * math.pow(10, 8)) / wavelength
            # Determines the total energy of an electron
            tot_energy = (6.62607004 * math.pow(10, -34)) * frequency
            # Kinetic energy is leftover energy from breaking off of surface of metal.
            # If its positive, it has escaped the metal surface
            kin_energy = tot_energy - current_metal.work_func
            # Creates a new Photon
            Photon.PhotonList.append(Photon((set_light_colour(wavelength)), kin_energy))
            # Sets LastEmitted to a value inversely proportional to intensity
            # Higher the intensity, the sooner the next photon with be released
            Photon.LastEmitted = math.ceil((1/intensity) * 100)
        else:
            # If timer not yet at 0, decrement it
            Photon.LastEmitted -= 1


# Given a string name, finds the first metal in the MetalList that has the same name
# Returns that metal object
def find_metal(name):
    new_metal = None
    for m in Metal.MetalList:
        if name == m.name:
            new_metal = m
    return new_metal


# Loads any custom made metals the user has previously created
# Loads it from data/custom_metals.dat - a binary file
# Parameter: drop - A DropDown object to add the metals to
def load_custom_metals(drop):
    # Tries to open the file, if it can't, catches exception and tells user
    i = 0
    try:
        f = open("data/custom_metals.dat", "rb")
        # Once the file is open, keeps trying to read it until it reaches the end of the file
        while 1:
            try:
                # Uses the pickle module to deserialised the Metal object in the file
                new_metal = pickle.load(f)
                # Adds the metal's name to the MetalNames list
                Metal.MetalNames.append(new_metal.name)
                # Adds the custom metal to the drop-down list
                drop = add_new_metal(new_metal, drop)
            except (EOFError, pickle.UnpicklingError):
                break
        # Closes the file to prevent using unnecessary memory
        f.close()
    except FileNotFoundError:
        print("ERROR: Cannot find data/custom_metals.dat")
    # Returns the modified DropDown item
    return drop


# Adds a new metal object to the MetalList and updates the dropdown box that stores the metals
def add_new_metal(new_metal, drop):
    Metal.MetalList.append(new_metal)
    drop.data = Metal.MetalNames
    drop.options = drop.data
    return drop


# Calculates the alpha value for the colour of the light
# Takes in a wavelength between 100 and 850
# And an intensity between 0 and 100
def set_light_alpha(wavelength, intensity):
    # wMod is the modifier to the alpha that the wavelength causes
    w_mod = 1
    wavelength = wavelength * math.pow(10, 9)
    # If no light, fully transparent
    if intensity == 0:
        return 0
    else:
        # If the wavelength is between 350 and 300 nm, wMod decreases as wavelength does
        if wavelength < 350:
            if wavelength > 300:
                w_mod = 1 - ((350 - wavelength) / 50)
            else:
                # If wavelength below 300nm it's fully transparent as its below wavelength of visible light
                w_mod = 0
        # If the wavelength is between 750 and 800nm, wMod decreases as wavelength increases
        elif wavelength > 750:
            if wavelength < 800:
                w_mod = (800 - wavelength) / 50
            else:
                # If wavelength is above 800nm, it's fully transparent as its above wavelength of visible light
                w_mod = 0
        # alpha is capped at 128 (half of opaque value). Is proportional to intensity and wMod
        alpha = 100 * (intensity / 100) * w_mod
        # Rounds alpha to integer
        alpha = round(alpha)
        return alpha


# Used in setting the colour of the light and photons
# Uses the tuples min_wavelength and max_wavelength
# These tuples are wavelength boundaries for specific colours
# Given a wavelength, finds the upper and lower bounds of it to find what colour it is
def set_min_max(wavelength):
    min_wavelength = 0
    max_wavelength = 0
    for i in range(len(wlValues) - 1):
        if wavelength <= wlValues[i]:
            min_wavelength = wlValues[i]
            max_wavelength = wlValues[i+1]
    return min_wavelength, max_wavelength


# Returns an RGB colour tuple given a wavelength
# Finds the upper and lower bounds of the colour the wavelength causes
# Sets the colour proportionally to how far the wavelength value is between the boundaries
# For example: if the wavelength is half way between the boundary between yellow and red
# The colour is half-way between yellow and orange
def set_light_colour(wavelength):
    wavelength = wavelength * math.pow(10, 9)
    min_wavelength, max_wavelength = set_min_max(wavelength)
    # In this system, there are 3 colour variables, R G and B
    # One will always by 0, 1 will always be 255 (except for violet)
    # and the other will be var_colour
    # var_colour is highest when the wavelength is at the upper boundary and at lowest at lower boundary
    var_colour = round(((wavelength - min_wavelength) / (max_wavelength - min_wavelength)) * 255)
    r = 0
    g = 0
    b = 0
    # If ir to red
    if min_wavelength == wlValues[0]:
        r = 255
    # If red to yellow
    elif min_wavelength == wlValues[1]:
        r = 255
        g = var_colour
    # If yellow to green
    elif min_wavelength == wlValues[2]:
        r = 255 - var_colour
        g = 255
    # If green to cyan
    elif min_wavelength == wlValues[3]:
        g = 255
        b = var_colour
    # If cyan to blue
    elif min_wavelength == wlValues[4]:
        g = 255 - var_colour
        b = 255
    # If blue to purple
    elif min_wavelength == wlValues[5]:
        r = round((var_colour / 255) * 180)
        b = 255
    # If purple to UV
    elif min_wavelength == wlValues[6]:
        r = 180
        b = 255
    return r, g, b


# Loads from settings.dat
# Reads a boolean from the file that shows individual photons when True
def load_settings():
    # Checkbox holds value of the boolean in the binary file
    # Set to True by default
    checkbox = True
    try:
        # Opens the settings.dsy file, if it can't tells the user
        f = open("data/settings.dat", "rb")
        # Deserialises the boolean value saved to the file
        checkbox = pickle.load(f)
        # Closes the file to prevent unneeded memory use
        f.close()
    except(EOFError, pickle.UnpicklingError):
        print("Error reading settings.dat")
        # If file can't be read, checkbox is set to True by default
    except(FileNotFoundError):
        print("settings.dat is missing, creasing a new one")
        f = open("data/settings.dat", "wb")
        # Serialises boolean value of True as a default
        pickle.dump(True, f)
        # Closes file to prevent unnecessary memory usage
        f.close()
    return checkbox


# Deletes the contents of file f
def delete_file(f):
    f.seek(0)
    f.truncate()

# The main game code is run here
def game_loop():
    # Creating the loop boolean, this is false until the game exits
    game_exit = False

    # Starting value definitions
    wavelength = 0
    intensity = 0

    # Appends default metals to the metal list
    Metal.MetalList.append(Metal("Sodium", 3.65 * math.pow(10, -19), (100, 100, 100)))
    Metal.MetalList.append(Metal("Copper", 7.53 * math.pow(10, -19), (145, 88, 4)))
    Metal.MetalList.append(Metal("Zinc", 6.89 * math.pow(10, -19), (185, 195, 185)))
    Metal.MetalList.append(Metal("Magnesium", 5.90 * math.pow(10, -19), (205, 205, 205)))
    # Sets starting metal to the first one in the list (sodium)
    current_metal = Metal.MetalList[0]

    # Defines the fonts that the program will use for drawing text
    my_font = pygame.font.Font(None, 32)
    small_font = pygame.font.Font(None, 25)

    # Text objects used to describe the different GUI elements
    wave_txt = my_font.render("Wavelength: ", 1, (0, 0, 0))
    wave_txt2 = my_font.render("nm", 1, (0, 0, 0))
    intensity_txt = my_font.render("Intensity: ", 1, black)
    intensity_txt2 = my_font.render("%", 1, black)
    metal_txt = my_font.render("Metal: ", 1, black)
    stop_txt = my_font.render("Stopping Voltage: ", 1, black)
    stop_txt2 = my_font.render("V", 1, black)

    # Rectangles on left and right to represent metals
    left_rect = MetalRect(10, 400, 50, 150)
    right_rect = MetalRect(740, 400, 50, 150)

    # Wavelength Slider bar creation
    wv_slider = Slider(150, 5, 200, 25, small_font, (100, 850))
    # Setting default wavelength
    wavelength = wv_slider.get_pos()

    # Intensity slider bar creation
    int_slider = Slider(150, 40, 200, 25, small_font, (0, 100))
    # Setting default intensity
    intensity = int_slider.get_pos()
    # Stopping voltage slider creation
    stop_slider = Slider(300, 550, 200, 25, small_font, (-3, 3), 0.5, 1)
    stop_voltage = stop_slider.get_pos()
    # Dropdown menu creation
    drop = DropDown(90, 90, 120, 25, Metal.MetalNames, my_font)
    # Loads custom metals from the file
    drop = load_custom_metals(drop)
    # 'Create new metal' button creation
    btn = Button(250, 90, my_font, "Create New Metal")
    # Adding electron speed text to screen
    speed_obj = my_font.render("Average speed: 0 ms^-1", 1, (0, 0, 0))

    # Settings button
    settings_btn = ImageButton(730, 10, my_font, "options")

    # Adding buttons to save and load settings
    save_button = Button(270, 130, my_font, "Save Values")
    load_button = Button(270, 160, my_font, "Load Values")

    # Creating surface for transparent light texture
    surf = pygame.Surface((display_width, display_height), pygame.SRCALPHA)
    surf.set_alpha(set_light_alpha(wavelength, intensity))

    # Image for the lamp
    lamp_img = pygame.image.load("img/lamp.png")

    # Creating menu
    cnm_menu = Menu(200, 200, 450, 280, my_font, "Create New Metal")
    # Creating text box to add to menu
    menu_name_txt = Textbox(80, 5, 200, 25, my_font, ["|"], 15)
    # Tuple of values containing each letter of the alphabet
    # Used for the 'blocked characters' for a text entry box, also contains symbols
    alphabet = ("a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u",
                "v", "w", "x", "y", "z", "!", "\"", "£", "$", "%", "^", "&", "*", "(", ")", "_", "-", "+", "=", "?", "\\", "|", "<", ">", "{", "}", "[", "]", "/", ",")
    # Creates a text entry box for the work function, does not allow alphabet characters
    menu_work_txt = Textbox(170, 40, 80, 25, my_font, alphabet, 5)
    # Creates a button to close the 'Create New Metal' menu and add the metal created
    menu_ok_button = Button(390, 230, my_font, "Add")
    # Creates a button to close the 'Create New Metal' menu without adding a metal
    menu_close_button = Button(10, 230, my_font, "Close")
    # Creating RGB elements
    r_slider = Slider(80, 90, 200, 25, my_font, (0, 255), 0)
    g_slider = Slider(80, 130, 200, 25, my_font, (0, 255), 0)
    b_slider = Slider(80, 170, 200, 25, my_font, (0, 255), 0)
    # Sets starting values for each slider
    r = round(r_slider.get_pos())
    g = round(g_slider.get_pos())
    b = round(b_slider.get_pos())
    # Creates a colour patch object for displaying the colour from the 3 sliders
    patch = ColourPatch(330, 110, 100, 100, my_font, (r, g, b))
    # Creating checkbox to allow user to choose to save the file
    checkbox = Checkbox(140, 200, my_font)

    # Adding menu elements to menu, for explanation, see Menu object in dan_gui.py
    # Adding textboxes to menu
    cnm_menu.add(menu_name_txt)
    cnm_menu.add(menu_work_txt)
    # Add RGB sliders
    cnm_menu.add(r_slider)
    cnm_menu.add(g_slider)
    cnm_menu.add(b_slider)
    cnm_menu.add(patch)
    # Adding buttons to menu
    cnm_menu.add(menu_ok_button)
    cnm_menu.add(menu_close_button)
    # Adding Checkbox
    cnm_menu.add(checkbox)
    # Adding text
    cnm_menu.add_text("Name: ", (5, 5))
    cnm_menu.add_text("Work Function: ", (5, 40))
    cnm_menu.add_text("x 10^-19 J", (260, 40))
    cnm_menu.add_text("Colour: ", (5, 70))
    cnm_menu.add_text("Red: ", (5, 100))
    cnm_menu.add_text("Green: ", (5, 140))
    cnm_menu.add_text("Blue: ", (5, 180))
    cnm_menu.add_text("Save to file?", (5, 200))

    # Making Settings Menu
    settings_menu = Menu(200, 200, 450, 250, my_font, "Settings")
    # Creating objects for settings menu
    photon_checkbox = Checkbox(200, 10, my_font)
    # Initialises variable to hold whether checkbox is on or off
    photon_checkbox.on = True
    photon_draw = photon_checkbox.on
    # Storage Settings Button
    store_btn = Button(10, 50, my_font, "Storage Settings")
    # Creating button for saving settings
    save_set_btn = Button(10, 200, my_font, "Save Settings")
    # Adding Elements to Settings menu
    settings_menu.add(photon_checkbox)
    settings_menu.add(store_btn)
    settings_menu.add(save_set_btn)
    # Adding text to settings menu
    settings_menu.add_text("Show Photons:", (5, 10))

    # Creates Storage Settings Menu
    store_menu = Menu(150, 200, 600, 250, my_font, "Storage Settings")
    # Creating objects for menu
    clear_v_btn = Button(10, 30, my_font, "Clear File")
    clear_c_btn = Button(10, 100, my_font, "Clear File")
    back_btn = Button(10, 160, my_font, "Back")
    # Adding objects to menu
    store_menu.add(clear_v_btn)
    store_menu.add(clear_c_btn)
    store_menu.add(back_btn)
    # Adding text to menu
    store_menu.add_text("values.dat - Holds data of saved values", (10, 10))
    store_menu.add_text("custom_metals.dat - Holds data of saved custom metals", (10, 80))

    # Last thing: load in settings
    photon_checkbox.on = load_settings()
    photon_draw = photon_checkbox.on

    # All code in this loop runs 30 times a second until the program is closed
    while not game_exit:
        # This gets all events pygame detects in one list
        events = pygame.event.get()
        # Gets the position as a pair of co-ords of the mouse in the current frame
        x, y = pygame.mouse.get_pos()

        # Updates the pointer for each of the sliders
        wv_slider.update(x)
        int_slider.update(x)
        stop_slider.update(x)
        # Checks if buttons are clicked with built in delay to prevent accidental muliple clicks
        btn.update(x, y)
        save_button.update(x, y)
        load_button.update(x, y)
        settings_btn.update(x, y)
        # Updates all elements in cnm menu - see Menu object in dan_Gui.py for explanation
        cnm_menu.update(x, y)
        # Updates colours in RGB slider base on slider position
        r = round(r_slider.get_pos())
        g = round(g_slider.get_pos())
        b = round(b_slider.get_pos())
        # Updates colour of ColourPatch
        patch.rgb = (r, g, b)
        # Updates elements in settings menu
        settings_menu.update(x, y)
        # Updates elements in storage menu
        store_menu.update(x, y)

        # Input management
        # Checks if each event in the events list matches certain types
        for event in events:
            # Checking for mouse clicked, gives position
            if event.type == pygame.MOUSEBUTTONDOWN:
                # Check if the drop down box is changed
                changed = drop.on_click(x, y)
                # If it has been changed then the current_metal is set to the metal selected by the drop down box
                if changed:
                    name = drop.data[drop.current_opt]
                    current_metal = find_metal(name)
                # Passes mouse co-ords onto sliders when click registered
                wv_slider.on_click(x, y)
                int_slider.on_click(x, y)
                stop_slider.on_click(x, y)
                # passes mouse co-ords onto buttons when clicked
                btn.on_click(x, y)
                save_button.on_click(x, y)
                load_button.on_click(x, y)
                settings_btn.on_click(x, y)
                # Checks clicks in the menus if open
                if cnm_menu.visible:
                    cnm_menu.on_click(x, y)
                elif settings_menu.visible:
                    settings_menu.on_click(x, y)
                elif store_menu.visible:
                    store_menu.on_click(x, y)

            # Checking for mouse unclicked
            if event.type == pygame.MOUSEBUTTONUP:
                # Triggers the sliders' methods for when a mouse is unclicked
                wv_slider.on_unclick()
                int_slider.on_unclick()
                stop_slider.on_unclick()
                # When unclicked, triggers methods in buttons and menus
                btn.on_unclick()
                save_button.on_unclick()
                load_button.on_unclick()
                cnm_menu.on_unclick()
                settings_btn.on_unclick()
                settings_menu.on_unclick()
                store_menu.on_unclick()

            # Checking for any keyboard key being pressed
            if event.type == pygame.KEYDOWN:
                # If the menu is visible, trigger appropriate method in menu
                if cnm_menu.visible:
                    cnm_menu.on_char_typed(event.key)

            # Checking for key being unpressed
            if event.type == pygame.KEYUP:
                # If the menu is visible, trigger appropriate method in menu
                if cnm_menu.visible:
                    cnm_menu.on_key_up(event.key)
                
            # Checking for exit, in event of exit event, the game closes and the loop stops
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
                game_exit = True

        # Checking if each button has been clicked

        # Checks if open menu button has been clicked
        # Opens 'Create new Metal' menu if no other open menus
        if btn.clicked and not settings_menu.visible:
            cnm_menu.visible = True
        # Checks if settings menu button has been clicked
        # Opens Settings menu if no other open menus
        if settings_btn.clicked and not cnm_menu.visible:
            settings_menu.visible = True

        # Checks if add metal menu button is pressed and text boxes are filled in
        if menu_ok_button.clicked and menu_name_txt.text != "" and menu_work_txt != "":
            # Closes 'Create New Metal' menu
            cnm_menu.visible = False
            # Creates a new Metal object and adds to drop-down box
            new_metal = Metal(menu_name_txt.text, get_float_from_string(menu_work_txt.text) * math.pow(10, -19), colour)
            drop = add_new_metal(new_metal, drop)
            # If user chose to save the metal to a file
            if checkbox.on:
                # Opens the file and tells user if it can't
                try:
                    f = open("data/custom_metals.dat", "ab")
                    # Serialises the metal object and writes it to the file
                    pickle.dump(new_metal, f)
                    # Closes the file to prevent unneeded memory use
                    f.close()
                except FileNotFoundError:
                    print("Cannot open data/custom_metals.dat")
            # Resets the text entry boxes to be empty
            menu_name_txt.text = ""
            menu_work_txt.text = ""

        # Checks if close menu button has been pressed, closes menu and resets text fields
        if menu_close_button.clicked:
            cnm_menu.visible = False
            menu_name_txt.text = ""
            menu_work_txt.text = ""

        # Checks if save values button is pressed
        if save_button.clicked:
            # Tries to open the values file, if it can't, tells the user
            try:
                f = open("data/values.dat", "wb")
                pickle.dump(wv_slider.pointer, f)
                pickle.dump(int_slider.pointer, f)
                pickle.dump(drop.current_opt, f)
                f.close()
            except FileNotFoundError:
                print("Cannot open data/value.dat")

        # Checks if load settings button is pressed
        if load_button.clicked:
            # Tries to open values file, tells user if can;t
            try:
                f = open("data/values.dat", "rb")
                try:
                    # Sets slider values to deserialised values from file
                    wv_slider.pointer = pickle.load(f)
                    int_slider.pointer = pickle.load(f)
                    # Tries to set current option of the drop-down box
                    # Will default to first option if fails to load
                    try:
                        drop.current_opt = pickle.load(f)
                        drop.change_text(drop.data[drop.current_opt])
                    except IndexError:
                        drop.current_opt = 0
                        drop.change_text(drop.data[drop.current_opt])
                    # Updates current_metal to the metal loaded from file
                    current_metal = find_metal(drop.data[drop.current_opt])
                except(EOFError, pickle.UnpicklingError):
                    pass
                # Closes file once done to prevent unnecessary memory usage
                f.close()
            except FileNotFoundError:
                print("Cannot open values.dat")

        # Checks if the 'save settings' button (from settings menu) has been pressed
        if save_set_btn.clicked:
            # Try to open settings file, if can't, tell user
            try:
                f = open("data/settings.dat", "wb")
                # Serialises boolean value of whether checkbox is on
                pickle.dump(photon_checkbox.on, f)
                # Closes file to prevent unnecessary memory usage
                f.close()
            except FileNotFoundError:
                print("Cannot load settings.dat")
            # Closes settings menu
            settings_menu.visible = False

        # Checks if 'Open storage menu' button (in settings menu) is clicked
        if store_btn.clicked:
            # Closes settings menu
            settings_menu.visible = False
            # Opens storage menu
            store_menu.visible = True

        # Checks if the button for clearing the 'values.dat' file is clicked
        if clear_v_btn.clicked:
            # Opens the values file
            with open("data/values.dat", "wb") as f:
                # Deletes contents of the file
                delete_file(f)
            # Greys out the button to prevent being used again
            clear_v_btn.grey = True

        # Checks if the button for clearing the 'custom_metals.dat' file is clicked
        if clear_c_btn.clicked:
            # Opens custom metals file
            with open("data/custom_metals.dat", "wb") as f:
                # Deletes the contents of the file
                delete_file(f)
            # Greys out the button prevent being used again
            clear_c_btn.grey = True

        # If the back button (in the storage menu) is clicked
        if back_btn.clicked:
            # Opens the settings menu
            settings_menu.visible = True
            # Closes the storage menu
            store_menu.visible = False
        
        # Updates rgb values for colour patch on cnm Menu if its open
        if cnm_menu.visible:
            colour = (r, g, b)
        # Updates the boolean for whether protons should be drawn if settings menu is open
        elif settings_menu.visible:
            photon_draw = photon_checkbox.on

        # ALL CALCULATIONS BELOW HERE
        # Gets the wavelength from the slider
        wavelength = wv_slider.get_pos()
        # Multiplies it to be the correct order of magnitude (nanometres)
        wavelength = wavelength * math.pow(10, -9)
        # Gets RGB values for light according to wavelength
        r, g, b = set_light_colour(wavelength)

        # Sets the intensity to the 2nd slider's value
        intensity = int_slider.get_pos()

        # Gets stopping voltage
        stop_voltage = stop_slider.get_pos()

        # Emits a photon if needed
        emit_photon(current_metal, intensity, wavelength)

        # Draws white over previous frame
        screen.fill(white)
        # ALL DRAWING BELOW HERE
        # For every photon in the PhotonList
        for photon in Photon.PhotonList:
            # Moves the photon's x and y co-ords
            photon.move()
            # #Draws the photon if the setting for drawing photons is enabled
            if photon_draw:
                photon.draw(screen)
            # Checks if photon has hit left metal plate
            photon.check_collision(left_rect.rect, stop_voltage)

        # Draw Electrons and calculate their average speed using their kinetic energy
        total_ke = 0
        for electron in Electron.ElectronList:
            # Adds each electron's kinetic energy to the total
            total_ke += electron.kinEnergy
            electron.move()
            electron.draw(screen)
            electron.check_pos(right_rect.rect)
        # If the ElectronList is not empty
        if len(Electron.ElectronList) > 0:
            # Calculates average kinetic energy of all electrons
            average_ke = total_ke / len(Electron.ElectronList)
            # Converts kinetic energy to speed
            speed = round(math.sqrt((2*average_ke)/Electron.Mass))
            # Creates a pygame Text object for rendering the speed
            speed_obj = small_font.render(("Average Speed: " + str(speed) + " ms^-1"), 1, black)

        # Draws background for wavelength, intensity and current metal selectors
        pygame.draw.rect(screen, lightGrey, (0, 0, 450, 200))
        # Draws border around bottom and right sides of box
        pygame.draw.lines(screen, black, False, ((0, 200), (450, 200), (450, 0)), 2)
        # Drawing average speed
        screen.blit(speed_obj, (5, 150))
        # Left rectangle
        left_rect.draw(screen, current_metal.colour)
        # Right rectangle
        right_rect.draw(screen, grey)
        # Wavelength slider prompt
        screen.blit(wave_txt, (5, 5))
        # Wavelength slider
        wv_slider.draw(screen)
        # Wavelength slider suffix
        screen.blit(wave_txt2, (400, 5))
        # Intensity slider prompt
        screen.blit(intensity_txt, (5, 40))
        # Intensity slider suffix
        screen.blit(intensity_txt2, (400, 40))
        # Draw intensity slider
        int_slider.draw(screen)
        # Stopping voltage slider
        stop_slider.draw(screen)
        # Stopping voltage slider prompt
        screen.blit(stop_txt, (100, 550))
        # Stopping voltage slider suffix
        screen.blit(stop_txt2, (540, 550))
        # Metal Text
        screen.blit(metal_txt, (5, 90))
        # Drop down box
        drop.draw(screen)
        # Draw button that opens 'Create new metal' menu
        btn.draw(screen)
        # Draws save and load buttons to screen
        save_button.draw(screen)
        load_button.draw(screen)
        # Draw settings button
        settings_btn.draw(screen)

        # Draws light from lamp to screen
        # Gets alpha (transparency) value for light
        alpha = set_light_alpha(wavelength, intensity)
        # Combines colour with alpha in 1 tuple
        light_colour = (r, g, b, alpha)
        # Draws light to transparency enabled surface
        pygame.draw.polygon(surf, light_colour, ((60, 400), (60, 550), (700, 380), (512, 202)))
        # Draws transparent surface to screen
        screen.blit(surf, (0, 0))
        # Draws lamp image
        screen.blit(lamp_img, (500, 150))

        # Draws menus
        cnm_menu.draw(screen)
        settings_menu.draw(screen)
        store_menu.draw(screen)

        # Updates the display
        pygame.display.update()

        # Makes the program wait so that the main loop only runs 30 times a second
        clock.tick(30)


# Calls the main loop subroutine to start
if __name__ == "__main__":
    game_loop()


ERROR: Cannot find data/custom_metals.dat
Could not find file at img/options.png


AttributeError: 'ImageButton' object has no attribute 'image'